# Проект Sumsung

In [1]:
!pip install -r requirements.txt

In [9]:
import pandas as pd
import numpy as  np
from yandex_geocoder import Client
from geopy.distance import geodesic
import ast
import re
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

# Initialize the geocoder client
client = Client("0f648b4c-4790-4e45-9195-b016a7729abf")

# User's coordinates
a = input('Введите свой адрес в формате улица дом:')
print('Адрес:', a)
coordinates = client.coordinates(a)
coordinate = re.sub(r'[A-z]+\s?', '', str(coordinates)).strip()
nums = re.findall(r'\d*\.\d+|\d+', coordinate) 
nums = [float(i) for i in nums]
m = nums[0]
nums[0]= nums[1]
nums[1] = m
print('Координаты', nums)

# Read and process coordinates data
cord = pd.read_csv('coordinates.csv')
coord_list = [ast.literal_eval(coord) for coord in cord['Coordinates']]

# Calculate distances
distances = [geodesic(coordinates, coord).km for coord in coord_list]

# Now distances[i] corresponds to the distance to the i-th coordinate

# Schools' attributes (replace these with your actual data)
workload = [(random.random() * (101 - 51) + 51) for _ in range(497)]  # workload percentages for each school
feedback = [random.random() * (5 - 2) + 2 for _ in range(497)]   # feedback scores for each school


Введите свой адрес в формате улица дом:Путевой проезд, 8
Адрес: Путевой проезд, 8
Координаты [55.874769, 37.581119]


In [1]:
import ipywidgets as widgets
from IPython.display import display

# Ввод данных пользователем через слайдеры
# важность
print('Оцените приоритетность близости школы от 0 до 10:')
slider1 = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='',
    value=0)
display(slider1)

print('Оцените приоритетность заполненности школы от 0 до 10:')
slider2 = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='',
    value=0)
display(slider2)

print('Оцените приоритетность рейтинга школы от 0 до 10:')
slider3 = widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='',
    value=0)
display(slider3)

Оцените приоритетность близости школы от 0 до 10:


IntSlider(value=0, max=10)

Оцените приоритетность заполненности школы от 0 до 10:


IntSlider(value=0, max=10)

Оцените приоритетность рейтинга школы от 0 до 10:


IntSlider(value=0, max=10)

In [11]:
d = slider1.value/10
w = slider2.value/10
f = slider3.value/10
# Importance of attributes
weights = [d, w, f]


# Parameters for schools
X = np.array([distances, workload, [5 - f for f in feedback]]).T
y = list(range(len(distances)))

# Normalize parameters
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

X_w = X_scaled * np.array(weights)

# Train classifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(X_w, y)
#print('Лучший результат', neigh.predict_proba([[0,0,0]]))
pred_index = neigh.predict([[0,0,0]])
#Вывод результатов
sch = pd.read_csv('school_data.csv')
Name = sch['ShortName'][pred_index[0]]
predicted_address = cord['LegalAddress'][pred_index[0]]
predicted_cord = cord['Coordinates'][pred_index[0]]
n = re.findall(r'\d*\.\d+|\d+', predicted_cord) 
n = [float(i) for i in n]
b = n[0]
n[0]= n[1]
n[1] = b
SchoolsFeedback = feedback[pred_index[0]]
SchoolsWorkload = workload[pred_index[0]]
print('Адрес школы:', predicted_address)
#print('Predicted Coordinates:', predicted_cord, n)
print('Рейтинг школы:',SchoolsFeedback)
print('Заполненность школы:',SchoolsWorkload)
print(Name)

Адрес школы: 127540, ГОРОД МОСКВА, ПРОЕЗД КЕРАМИЧЕСКИЙ, ДОМ 47А
Рейтинг школы: 4.8498025659648185
Заполненность школы: 86.8242796638155
ГБОУ школа № 1794


In [12]:
#Создание карты
import folium

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium DivIcon plugin
from folium.features import DivIcon

# generate map
Moscow_map = folium.Map(
    location = nums, # За центр берётся локация ползователя
    zoom_start = 17
)


folium.Marker(n,   # координаты школы
popup = Name, 
icon=folium.Icon(color="red", icon="info-sign")
).add_to(Moscow_map)
folium.Marker(nums,   # координаты пользователя
popup = 'Ваш дом').add_to(Moscow_map)

# display map
Moscow_map